In [1]:
import sys
#sys.path.insert(0,'./src/')
#simulator_path = "../covid_de/sim/" ##change simulatio path here 
sys.path.insert(0,'src/')
sys.path.insert(0, '../sib/')
simulator_path = "../covid_sim_giova/sim/" ##change simulatio path here 
sys.path.insert(0,simulator_path)

from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json, log, logging, loop_mtpp
from importlib import reload
import imp
from math import exp
import pickle
#sys.path.insert(0,'./src/loop_ranker')
from lib.mobilitysim import MobilitySimulator
import loop_mtpp
from mtpp_utils import contacts_cg


#logging
data_path = '../sib_test/results/MTPP/Isle_of_Wight/'

output_dir = data_path + "/output_pop20_site10/"
fold_out = Path(output_dir)
if not fold_out.exists():
    fold_out.mkdir(parents=True)

reload(log)
logger = log.setup_logger()


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


Log setup complete


### Mobility simulation

In [2]:
beta = 1.14 # from paper
country = 'GER'
with open(data_path + 'Isle_of_Wight_settings_pop10_site5.pk', 'rb') as fp:
    mob_kwargs = pickle.load(fp)

mob_kwargs["delta"] = 0.2554120904376099
T = 50
seed_mob = 1
seed = 1
t_unit = 24
t_res = 0.25 # drop contacts with a duration < t_res (in hours)
max_time = T * t_unit  
mob = MobilitySimulator(**mob_kwargs)
mob.verbose = True
out = mob.simulate(max_time=max_time, seed=seed_mob)
#contacts_df = pd.DataFrame(contacts_cg(mob, t_res, t_unit, first_filter = True),columns = ['i','j','t','deltat'])
N = mob.num_people 
#print(N)

Simulate mobility for 1200.00 time units... 

In [3]:
contacts_df = pd.DataFrame(contacts_cg(mob, t_res, t_unit, first_filter = True),columns = ['i','j','t','deltat'])

Check asymmetric contacts
All raw (i < j) contacts 1985768
0.0 %
25.18 %
50.36 %
75.54 %
Coarse-grained contacts 1981276


In [4]:
contacts_df = contacts_df.sort_values(by=["t","i","j"])

## Set testing and quarantine rules

In [5]:
#n_indiv=np.ceil(mob_kwargs['num_people_unscaled']/mob_kwargs['downsample_pop'])
n_seeds = {'expo': 2, 'iasy':2,'ipre':2} # select initial infected seeds
num_test_random = 0 #number of random tests per day
fraction_sym_obs = 0.5 #fraction of Symptomatic tested positive
initial_steps = 5 #starting time of intervention
delta_days = 1 # intervention every delta_days days (for the moment keep to 1)
#assert initial_steps % delta_days == 0
test_HH = False
quarantine_HH = True
adoption_fraction = 1.0

## Set the inference algorithm class

In [7]:
import sib,  scipy

from loop_ranker import sib_rank, sib_drop_rank, greedy_rank, dotd_rank, mean_field_rank, tracing_rank, winbp_rank, winbp_prob0_rank
from tqdm.notebook import tqdm
from scipy.stats import gamma
sib.set_num_threads(6)

import matplotlib.pyplot as plt

mu = 1/12
prob_seed = 1/N
prob_sus = 0.5
pautoinf = 1e-10
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-6
fp_rate = 0.0
fn_rate = 0.0

rankers = {}
rankers["dotd"] = dotd_rank.DotdRanker()
'''

rankers["MF"] = mean_field_rank.MeanFieldRanker(
                tau = 5,
                delta = 10,
                mu = mu,
                lamb = 1.0
                )
'''
rankers["tracing"] = tracing_rank.TracingRanker(
                 tau=5,
                 lamb=1.0
)

rankers["greedy"] = greedy_rank.GreedyRanker(
                 include_S = True,
                tau=10)


"""
rankers["winbp20prob0"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Uniform(1.0),
                                 prob_r = sib.Exponential(mu=mu),
                                 #prob_i = sib.PriorDiscrete(list(0.25 * abm_utils.gamma_pdf_array(T+1,6,2.5))),
                                 #prob_r = sib.PriorDiscrete(list(scipy.stats.gamma.sf(range(T+1), 10., scale=1.7452974337097158))),
                                 #prob_i = sib.Uniform(0.014),
                                 #prob_r = sib.Gamma(k = 10, mu = 1/1.7452974337097158),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 30,
                 maxit1 = 30,
                 damp0 = 0.5,
                 damp1 = 0.9,
                 tol = 1e-3,
                 memory_decay = 1e-2,
                 window_length = 20
)


rankers["no_intervention"] = None

rankers["winbp21_tau7"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),
                                 prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=7
)

rankers["winbp21_tau0"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),
                                 prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=0
)
"""

rankers["winbp21_uninf"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Uniform(1.0), 
                                prob_r = sib.Exponential(0.1), 
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=0
)



In [ ]:
rankers

In [ ]:
ress = {}
for num_test_algo in [200]: #number of test per day by ranking
    for seed in [1]:
        print(num_test_algo, seed)
        for s in list(rankers.keys()):
            data = {"algo":s}
            if s== "no_intervention":
                    res_s = loop_mtpp.free_mtpp(mob,
                    country  = country,
                    beta = beta,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_counts = n_seeds,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                     )
            else:
                    res_s = loop_mtpp.loop_mtpp(mob,
                    contacts_df,
                    rankers[s],
                    country  = country,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_steps = initial_steps, 
                    num_test_random = num_test_random,
                    num_test_algo = num_test_algo,
                    fraction_sym_obs = fraction_sym_obs,
                    initial_counts = n_seeds,
                    beta = beta,
                    test_HH = test_HH,
                    quarantine_HH = quarantine_HH,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                    save_every_iter = 1,
                    adoption_fraction = adoption_fraction
                 )
            ress[s] = res_s    
        del res_s


200 1
Starting with guys:  {'expo': [10569, 8248], 'iasy': [4355, 12084], 'ipre': [12801, 14142]}
t: 24.00 | 1200.00 hrs (50 d)

iteration.dotd: INFO     time:0
numexpr.utils: INFO     Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
numexpr.utils: INFO     Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
numexpr.utils: INFO     NumExpr defaulting to 8 threads.
iteration.dotd: INFO     number of unique contacts: 91364



[Reached max time: 24h (1d)]


iteration.dotd: INFO     True  : (S,I,R): (14188.0, 6.0, 0.0)
iteration.dotd: INFO     AUR_I : 0.328, prec100: 0, prec(1% of 14195): 0.00, prec5%: 0.00
iteration.dotd: INFO     sym: 0, results test algo (S,I,R): (0,0,0), infected test random: 0/0
iteration.dotd: INFO     ...quarantining 0 guys -> got 0 infected, 6 free as birds (+6)


t: 48.00 | 1200.00 hrs (50 d)

...

iteration.dotd: INFO     time:11
iteration.dotd: INFO     number of unique contacts: 103994
iteration.dotd: INFO     True  : (S,I,R): (10631.0, 3563.0, 0.0)
iteration.dotd: INFO     AUR_I : 0.488, prec100: 0, prec(1% of 13605): 0.00, prec5%: 0.15
iteration.dotd: INFO     sym: 64, results test algo (S,I,R): (158,42,0), infected test random: 0/0
iteration.dotd: INFO     ...quarantining 310 guys -> got 297 infected, 2828 free as birds (+821)


t: 305.03 | 1200.00 hrs (50 d)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



t: 312.00 | 1200.00 hrs (50 d)
[Reached max time: 312h (13d)]


iteration.dotd: INFO     time:12
iteration.dotd: INFO     number of unique contacts: 100434


In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(1,4,figsize = (25,5))
to_plot = ['I','infected_free','num_quarantined', "q_algo"]
for s in ress.keys():    
    for il,l in enumerate(to_plot):
        if s != "no_intervention":
            axs[il].plot(range(T),ress[s][l],label = s)
            if l!= "aurI":
                axs[il].set_yscale ("log") 
                axs[il].vlines(initial_steps,ymin = 1,ymax = 10**4,linestyle = "--",alpha = 0.5, color = "black")
            else:
                axs[il].vlines(initial_steps,ymin = 0,ymax = 1,linestyle = "--",alpha = 0.5, color = "black")
            
            axs[il].set_title(l,fontsize = 12)
    axs[0].legend(loc = 'upper left')
    
#fig.suptitle(f"num test algo {num_test_algo} - adoption_fraction {adoption_fraction}")
fig.show()
